In [8]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)
memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key='history',
    return_messages=True,
)
prompt = ChatPromptTemplate.from_messages(
    [
        ('system', 'You are a helpful AI talking to a human. '),
        MessagesPlaceholder(variable_name='history'),
        ('human', '{question}'),
    ]
)


def load_memory(input):
    print(input)
    return memory.load_memory_variables({})['history']


chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm


def invoke_chain(question):
    results = chain.invoke({'question': "My name is Flynn."}),
    memory.save_context({'input': question}, {'output': ' '.join([result.content for result in results])})
    print(results)
    
invoke_chain('My name is Flynn.')

{'question': 'My name is Flynn.'}
(AIMessage(content='Hello Flynn! How can I assist you today?'),)


In [9]:
invoke_chain('What is my name?')

{'question': 'My name is Flynn.'}
(AIMessage(content="Hello Flynn! It seems like you've mentioned your name twice. Is there anything specific you'd like to know or discuss today?"),)
